In [1]:
import cv2
import math
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

In [3]:
video_path = "./video/suyash_oneplus.mp4"

In [4]:
cap = cv2.VideoCapture(video_path)
ret,IMG = cap.read()
IMG = cv2.resize(IMG,(IMG.shape[1],IMG.shape[0]))

In [5]:
def get_distance(Cb_p,Cr_p,Cb_key,Cr_key,tola,tolb):
    temp = math.sqrt((Cb_key-Cb_p)**2 + (Cr_key-Cr_p)**2)
    if(temp < tola):
        return 0.0
    elif (temp < tolb):
        return (temp-tola)/(tolb-tola)
    else:
        return 1.0

In [6]:
ctr=0

In [7]:
def segment(orig,params,tola,tolb):
    Cb_key,Cr_key = params
    ycrcb_im = cv2.cvtColor(orig,cv2.COLOR_BGR2YCrCb)
    mask = np.zeros((orig.shape[0],orig.shape[1]),dtype=np.float32)
    for i in range(orig.shape[0]):
        for j in range(orig.shape[1]):
            mask[i,j] = get_distance(ycrcb_im[i,j][2],ycrcb_im[i,j][1],Cb_key,Cr_key,tola,tolb)
    mask = 1.0 - mask
    return mask

In [8]:
def get_params(img):
    ycrcb_img = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb).astype(np.float32)
    r = cv2.selectROI(img)
    cv2.destroyAllWindows()
    r = [int(x) for x in r]
    region = ycrcb_img[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    y_mean,Cr_mean,Cb_mean = np.mean(region,axis=(0,1))
    y_std,Cr_std,Cb_std = np.std(region,axis=(0,1))
    return [Cb_mean,Cr_mean]

In [9]:
params = get_params(IMG)

In [10]:
WHITE = np.ones((IMG.shape[0]//2,IMG.shape[1]//2,3),dtype=np.int8)*255

In [11]:
cap = cv2.VideoCapture(video_path)
while(True):
    ret,img = cap.read()
    if not ret:
        break
    img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
    mask = np.expand_dims(segment(img,params,12,20),-1)
    new_img = ((1-mask)*img+(mask)*WHITE).astype(np.uint8)
#     cv2.imshow("img",img)
#     cv2.imshow("masked",new_img)
    ctr += 1
    s= "%05d" %(ctr,)
    cv2.imwrite('ignore/temp/img'+s+'.png',new_img)
    ret = cv2.waitKey(1)
    if ret==27:
        break
cv2.destroyAllWindows()